# Task 3 – Construction the network

After successfully creating both the custom dataset and the dataloader, you need to create a
neural network, and use the data loader to feed the network. The architecture, complexity and
regularization are all up to you, but you need to justify your choices in comments. You are
more than welcome to replicate already known architectures or architectures we made during
the course, but you are NOT allowed to use any pretrained networks. You are also not
allowed to use any training data that is not included on ItsLearning.
Carefully consider which hyperparameters to test and strategically try to find the optimal
architecture for the task. In the comments, please describe your method for the optimization
and your choice of hyperparameters. Remember that there is an underlying competition, and
the highest accuracy wins. The competition will be measured based on the saved model, so
make sure to submit only the best one!

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
#from torchvision import datasets, transforms
#import seaborn as sns

In [10]:
rgb = 3
greyscale = 1
h, w = 256, 256

In [11]:
input_dim=(h,w)
channel_dim=greyscale

In [12]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA")
else:
    device = torch.device("cpu")
    print("Using CPU")

Using CPU


In [25]:
# From Task 2
import numpy as np
import glob
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
import torchvision.transforms as transforms

class CustomDataset(Dataset):
    def __init__(self, img_size, class_names, path=None, transformations=None, num_per_class: int = -1):
        self.img_size = img_size
        self.path = path
        self.num_per_class = num_per_class
        self.class_names = class_names
        self.transforms = transformations
        self.data = []
        self.labels = []

        if path:
            self.readImages()

        self.standard_transforms = transforms.Compose([
            transforms.ToTensor()
            ])

    def readImages(self):
        for id, class_name in self.class_names.items():
            print(f'Loading images from class: {id} : {class_name}')
            img_path = glob.glob(f'{self.path}{class_name}/*.jpg')
            if self.num_per_class > 0:
                img_path = img_path[:self.num_per_class]
            self.labels.extend([id] * len(img_path))
            for filename in img_path:
                img = Image.open(filename).convert('L')
                img = img.resize(self.img_size)
                self.data.append(img)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = self.data[idx]
        label = self.labels[idx]

        if self.transforms:
            img = self.transforms(img)
        else:
            img = self.standard_transforms(img)

        label = torch.tensor(label, dtype=torch.long)

        return img, label
    
    
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=20),
    transforms.RandomAffine(degrees=20, translate=(0.1, 0.1)),
    # transforms.v2.ScaleJitter(target_scale=img_size, scale_range=(0.9, 1.1)),
    transforms.RandomAutocontrast(p=0.25),
    # transforms.GaussianBlur(kernel_size=3, sigma=(0,2.0)),
    # transforms.GaussianNoise(kernel_size=3, std=0.1)
])

train_path = "./data/training/"
test_path = "./data/testing/"
validation_path = "./data/validation/"
img_size=(256,256)


class_names = [name[len(train_path):] for name in glob.glob(f'{train_path}*')]
class_names = dict(zip(range(len(class_names)), class_names))

train_dataset = CustomDataset(img_size=img_size, path=train_path, class_names=class_names, transformations=train_transform)
test_dataset = CustomDataset(img_size=img_size, path=test_path, class_names=class_names)
validation_dataset = CustomDataset(img_size=img_size, path=validation_path, class_names=class_names)
# We could add batches to the DataLoader
train_dataloader = DataLoader(train_dataset, shuffle=True)
test_dataloader = DataLoader(test_dataset, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, shuffle=True)

Loading images from class: 0 : pneumonia
Loading images from class: 1 : normal
Loading images from class: 0 : pneumonia
Loading images from class: 1 : normal
Loading images from class: 0 : pneumonia
Loading images from class: 1 : normal


In [33]:
train_dataset[0][0].shape

torch.Size([1, 256, 256])

In [34]:
class group_9(nn.Module):
    def __init__(self):
        super(group_9, self).__init__()        

        # Convolutional Block 1
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3)  # Adjust `channel_dim` to 1 if needed
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        
        # Convolutional Block 2
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        # Convolutional Block 3
        self.conv5 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        self.conv6 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3)
        self.maxpool3 = nn.MaxPool2d(kernel_size=2)
        
        # Fully Connected Layers
        self.fc1 = nn.Linear(128 * 28 * 28, 128)  # Updated size
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        # Convolutional Block 1
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = self.maxpool1(x)
        
        # Convolutional Block 2
        x = torch.relu(self.conv3(x))
        x = torch.relu(self.conv4(x))
        x = self.maxpool2(x)
        
        # Convolutional Block 3
        x = torch.relu(self.conv5(x))
        x = torch.relu(self.conv6(x))
        x = self.maxpool3(x)
        
        # Flattening
        x = x.view(x.size(0), -1)
        
        # Fully Connected Layers
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

In [35]:
model = group_9()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

model.to(device)

group_9(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv6): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (maxpool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=100352, out_features=128, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)

In [36]:
def train(model, num_epochs: int = 10):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.005)
    model.to(device=device)
    losses = []

    for epoch in range(num_epochs):
        model.train()

        for data, targets in train_dataloader:
            data, targets = data.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(data)

            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            losses.append(loss.item())

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")

In [37]:
def test(model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, targets in test_dataloader:
            data, targets = data.to(device), targets.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

In [38]:
train(model=model, num_epochs=10)
test(model=model)

KeyboardInterrupt: 